In [1]:
from graphdatascience import GraphDataScience
from torch_geometric.data import Data, download_url
import torch
import torch.optim as optim
from torch_geometric.nn import TransE

In [2]:
gds = GraphDataScience("bolt://localhost:7687", auth=('neo4j', 'neo4jneo4j'), database="fb15k-237")

In [3]:
url = ('https://raw.githubusercontent.com/villmow/datasets_knowledge_embedding/master/FB15k-237')
raw_file_names = ['train.txt', 'valid.txt', 'test.txt']
raw_dir = './data_from_url'
for filename in raw_file_names:
    download_url(f'{url}/{filename}', raw_dir)

Using existing file train.txt
Using existing file valid.txt
Using existing file test.txt


In [4]:
def process():
    data_list_, node_dict_, rel_dict_ = [], {}, {}
    for file_name in raw_file_names:
        file_name_path = raw_dir + '/' + file_name
        with open(file_name_path, 'r') as f:
            data = [x.split('\t') for x in f.read().split('\n')[:-1]]

        edge_index = torch.empty((2, len(data)), dtype=torch.long)
        edge_type = torch.empty(len(data), dtype=torch.long)
        for i, (src, rel, dst) in enumerate(data):
            if src not in node_dict_:
                node_dict_[src] = len(node_dict_)
            if dst not in node_dict_:
                node_dict_[dst] = len(node_dict_)
            if rel not in rel_dict_:
                rel_dict_[rel] = len(rel_dict_)

            edge_index[0, i] = node_dict_[src]
            edge_index[1, i] = node_dict_[dst]
            edge_type[i] = rel_dict_[rel]

        data = Data(edge_index=edge_index, edge_type=edge_type)
        data_list_.append(data)

    for data in data_list_:
        data.num_nodes = len(node_dict_)

    return data_list_, node_dict_, rel_dict_

data_list, node_dict, rel_dict = process()

In [5]:
gds.run_cypher("CREATE CONSTRAINT entity_id FOR (e:Entity) REQUIRE e.id IS UNIQUE")

ClientError: {code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent constraint already exists, 'Constraint( id=4, name='entity_id', type='UNIQUENESS', schema=(:Entity {id}), ownedIndex=3 )'.}

In [6]:
rel_id_to_text_dict = {}
for k in rel_dict:
    text = k
    id = rel_dict[k]
    rel_id_to_text_dict[id] = text

In [7]:
print(data_list)
print(data_list[0].edge_index[0][1].item())
print(data_list[0].edge_type)

[Data(edge_index=[2, 272115], edge_type=[272115], num_nodes=14541), Data(edge_index=[2, 17535], edge_type=[17535], num_nodes=14541), Data(edge_index=[2, 20466], edge_type=[20466], num_nodes=14541)]
2
tensor([  0,   1,   2,  ..., 170,  30,  38])


In [9]:
def write_chunk(chunk_dict):
    gds.run_cypher(
            "UNWIND $nodes AS node CREATE (n:Entity {id: node[1], value: node[0]})",
            params={"nodes": list(chunk_dict.items())},
        )
    print(f"Written {len(chunk_dict)} elements...")

idx = 0
chunk_size = 1000
chunk_dict = {}
for k in node_dict:
    chunk_dict[k] = node_dict[k]
    idx += 1
    if idx % chunk_size == 0:
        write_chunk(chunk_dict)
        chunk_dict = {}
if len(chunk_dict) > 0:
    write_chunk(chunk_dict)
print(f"TOTAL records: {idx} from {len(node_dict)}")

Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 541 elements...
TOTAL records: 14541 from 14541


In [10]:
train_data = data_list[0]
val_data = data_list[1]
test_data = data_list[2]

In [11]:
def write_rel_chunk(ll:list, label):
    gds.run_cypher(
            "UNWIND $list AS l MATCH (e_s:Entity {id: l[0]}), (e_t:Entity {id: l[1]}) "+
            "CREATE (e_s)-["+label+" { rel_id: l[2], text: l[3] }]->(e_t)",
            params={"list": ll},
        )
    print(f"Written {len(ll)} elements...")


def create_rels(data:Data, label:str):
    idx = 0
    chunk_size = 1000
    chunk_list = []
    print("Writing " + label + " relationships")
    for i in range(data.num_edges):
        source = data.edge_index[0, i].item()
        target = data.edge_index[1, i].item()
        id = data.edge_type[i].item()
        text = rel_id_to_text_dict[id]
        l = [source, target, id, text]
        chunk_list.append(l)
        idx += 1
        if idx % chunk_size == 0:
            write_rel_chunk(chunk_list, label)
            chunk_list = []
    if len(chunk_list) > 0:
        write_rel_chunk(chunk_list, label)
    print(f"TOTAL records: {idx} from {data.num_edges}")

create_rels(test_data, ":TEST")
create_rels(val_data, ":VAL")
create_rels(train_data, ":TRAIN")

Writing :TEST relationships
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 466 elements...
TOTAL records: 20466 from 20466
Writing :VAL relationships
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 elements...
Written 1000 e

In [12]:
# Node: (:Entity {id:int, value:str})
# Edge: [:(TRAIN|TEST|VAL) {rel_id:int, text:str}]

In [13]:
def get_data_from_db(edge_label):
    node_projection = {"Entity": {"properties": "id"}}
    relationship_projection = {edge_label : {"orientation": "NATURAL", "properties": "rel_id"}}
    G, result = gds.graph.project(
        "fb15k-graph-t"+edge_label,
        node_projection,
        relationship_projection,
    )
    print(f"The projection took {result['projectMillis']} ms")

    # We can use convenience methods on `G` to check if the projection looks correct
    print(f"Graph '{G.name()}' node count: {G.node_count()}")
    print(f"Graph '{G.name()}' node labels: {G.node_labels()}")
    print(f"Graph '{G.name()}' relationship count: {G.relationship_count()}")

    return G

def get_whole_dataset():
    node_projection = {"Entity": {"properties": "id"}}
    relationship_projection = {
        "TRAIN" : {"orientation": "NATURAL", "properties": "rel_id"},
        "TEST" : {"orientation": "NATURAL", "properties": "rel_id"},
        "VAL" : {"orientation": "NATURAL", "properties": "rel_id"},
    }
    G, result = gds.graph.project(
        "fb15k-graph-whole",
        node_projection,
        relationship_projection,
    )
    print(f"The projection took {result['projectMillis']} ms")

    # We can use convenience methods on `G` to check if the projection looks correct
    print(f"Graph '{G.name()}' node count: {G.node_count()}")
    print(f"Graph '{G.name()}' node labels: {G.node_labels()}")
    print(f"Graph '{G.name()}' relationship count: {G.relationship_count()}")

    return G

In [14]:
train_db_data_G = get_data_from_db("TRAIN")
test_db_data_G = get_data_from_db("TEST")
val_db_data_G = get_data_from_db("VAL")
db_data_G = get_whole_dataset()

The projection took 973 ms
Graph 'fb15k-graph-tTRAIN' node count: 14541
Graph 'fb15k-graph-tTRAIN' node labels: ['Entity']
Graph 'fb15k-graph-tTRAIN' relationship count: 272115
The projection took 29 ms
Graph 'fb15k-graph-tTEST' node count: 14541
Graph 'fb15k-graph-tTEST' node labels: ['Entity']
Graph 'fb15k-graph-tTEST' relationship count: 20466
The projection took 11 ms
Graph 'fb15k-graph-tVAL' node count: 14541
Graph 'fb15k-graph-tVAL' node labels: ['Entity']
Graph 'fb15k-graph-tVAL' relationship count: 17535
The projection took 54 ms
Graph 'fb15k-graph-whole' node count: 14541
Graph 'fb15k-graph-whole' node labels: ['Entity']
Graph 'fb15k-graph-whole' relationship count: 310116


In [15]:
# gds.graph.drop(train_db_data_G)
# gds.graph.drop(test_db_data_G)
# gds.graph.drop(val_db_data_G)

In [16]:
print(db_data_G)

Graph(name=fb15k-graph-whole, node_count=14541, relationship_count=310116)


In [17]:
node_properties = gds.graph.nodeProperties.stream(
    db_data_G,
    ["id"],
    separate_property_columns=True,
)
print(node_properties)

       nodeId    id
0       10001  9690
1       10002  9691
2       10003  9692
3       10004  9693
4       10005  9694
...       ...   ...
14536    9996  9685
14537    9997  9686
14538    9998  9687
14539    9999  9688
14540   10000  9689

[14541 rows x 2 columns]


In [41]:
nodeId_to_id = dict(zip(node_properties.nodeId, node_properties.id))
id_to_nodeId = dict(zip(node_properties.id, node_properties.nodeId))

In [19]:
sample_topology_df = gds.beta.graph.relationships.stream(db_data_G)
# Let's see what we got:
display(sample_topology_df)

,sourceNodeId,targetNodeId,relationshipType
0,1,3,TRAIN
1,1,33,TRAIN
2,1,171,TRAIN
3,1,688,TRAIN
4,1,1663,TRAIN
...,...,...,...
310111,10000,124,TRAIN
310112,10000,124,TRAIN
310113,10000,367,TRAIN
310114,10000,425,TRAIN


In [20]:
rels_tmp = gds.graph.relationshipProperties.stream(db_data_G, ["rel_id"], separate_property_columns=True)
display(rels_tmp)
rels_tmp.rel_id.astype(int)
display(rels_tmp.sourceNodeId.map(lambda x: nodeId_to_id[x]))

,sourceNodeId,targetNodeId,relationshipType,rel_id
0,1,3,TRAIN,1.0
1,1,33,TRAIN,185.0
2,1,171,TRAIN,51.0
3,1,688,TRAIN,107.0
4,1,1663,TRAIN,201.0
...,...,...,...,...
310111,10000,124,TRAIN,4.0
310112,10000,124,TRAIN,5.0
310113,10000,367,TRAIN,80.0
310114,10000,425,TRAIN,4.0


0            2
1            2
2            2
3            2
4            2
          ... 
310111    9689
310112    9689
310113    9689
310114    9689
310115    9689
Name: sourceNodeId, Length: 310116, dtype: int64

In [21]:
topology = [rels_tmp.sourceNodeId.map(lambda x: nodeId_to_id[x]), rels_tmp.targetNodeId.map(lambda x: nodeId_to_id[x])]
edge_index = torch.tensor(topology, dtype=torch.long)
edge_type = torch.tensor(rels_tmp.rel_id.astype(int), dtype=torch.long)
display(edge_index)
display(edge_type)
data = Data(edge_index=edge_index, edge_type=edge_type)
data.num_nodes = len(nodeId_to_id)
print(data)

tensor([[   2,    2,    2,  ..., 9689, 9689, 9689],
        [   3,   32,  160,  ...,  344,  399,  399]])

tensor([  1, 185,  51,  ...,  80,   4,   5])

Data(edge_index=[2, 310116], edge_type=[310116], num_nodes=14541)


In [22]:
def create_tensor(graph):
    rels_tmp = gds.graph.relationshipProperties.stream(graph, ["rel_id"], separate_property_columns=True)
    topology = [rels_tmp.sourceNodeId.map(lambda x: nodeId_to_id[x]), rels_tmp.targetNodeId.map(lambda x: nodeId_to_id[x])]
    edge_index = torch.tensor(topology, dtype=torch.long)
    edge_type = torch.tensor(rels_tmp.rel_id.astype(int), dtype=torch.long)
    data = Data(edge_index=edge_index, edge_type=edge_type)
    data.num_nodes = len(nodeId_to_id)
    display(data)
    return data

train_tensor = create_tensor(train_db_data_G)
test_tensor = create_tensor(test_db_data_G)
val_tensor = create_tensor(val_db_data_G)

Data(edge_index=[2, 272115], edge_type=[272115], num_nodes=14541)

Data(edge_index=[2, 20466], edge_type=[20466], num_nodes=14541)

Data(edge_index=[2, 17535], edge_type=[17535], num_nodes=14541)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = TransE(
    num_nodes=train_tensor.num_nodes,
    num_relations=train_tensor.num_edge_types,
    hidden_channels=50,
).to(device)

loader = model.loader(
    head_index=train_tensor.edge_index[0],
    rel_type=train_tensor.edge_type,
    tail_index=train_tensor.edge_index[1],
    batch_size=1000,
    shuffle=True,
)

optimizer = optim.Adam(model.parameters(), lr=0.01)


def train():
    model.train()
    total_loss = total_examples = 0
    for head_index, rel_type, tail_index in loader:
        optimizer.zero_grad()
        loss = model.loss(head_index, rel_type, tail_index)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()
    return total_loss / total_examples


@torch.no_grad()
def test(data):
    model.eval()
    return model.test(
        head_index=data.edge_index[0],
        rel_type=data.edge_type,
        tail_index=data.edge_index[1],
        batch_size=20000,
        k=10,
    )


for epoch in range(1, 501):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    if epoch % 75 == 0:
        rank, hits = test(val_tensor)
        print(f'Epoch: {epoch:03d}, Val Mean Rank: {rank:.2f}, '
              f'Val Hits@10: {hits:.4f}')

print(model)
idx = torch.LongTensor([1])
print(model.rel_emb(idx))
rank, hits_at_10 = test(test_tensor)
print(f'Test Mean Rank: {rank:.2f}, Test Hits@10: {hits_at_10:.4f}')

In [ ]:
torch.save(model, "./model_501.pt")

In [ ]:
torch.save(model.state_dict(), "./model_501_st_dict")

In [23]:
model = torch.load("./model_501.pt")

In [24]:
rel_123 = 123
print(rel_id_to_text_dict[rel_123])
t = model.rel_emb(torch.LongTensor([rel_123]))
print(t)
rel_123_emb = t[0].tolist()
print(rel_123_emb)
print(len(rel_123_emb))

/base/popstra/celebrity/canoodled./base/popstra/canoodled/participant
tensor([[-0.1675,  0.0308, -0.1782,  0.1345, -0.0045,  0.0340,  0.0159, -0.3096,
         -0.0745, -0.0816, -0.1337, -0.0214,  0.2474,  0.0208,  0.0202, -0.2063,
          0.2327,  0.1317, -0.0562,  0.2252,  0.2233, -0.0494,  0.0689, -0.1421,
         -0.1578, -0.2542, -0.3715, -0.0021, -0.1611,  0.2791,  0.4206, -0.0711,
         -0.1338, -0.0318, -0.0076, -0.0900, -0.0040,  0.0135, -0.2083,  0.3131,
         -0.2107, -0.1513, -0.2384, -0.0324,  0.2208, -0.3213,  0.1846,  0.0350,
          0.0129, -0.3476]], grad_fn=<EmbeddingBackward0>)
[-0.1674661487340927, 0.030775610357522964, -0.17823708057403564, 0.13445991277694702, -0.004469443578273058, 0.03396214172244072, 0.015884431079030037, -0.30958986282348633, -0.07452814280986786, -0.08162254840135574, -0.1337374746799469, -0.021421784535050392, 0.24736957252025604, 0.020815536379814148, 0.020227091386914253, -0.2062523514032364, 0.23273737728595734, 0.1317358613014

In [25]:
print(model.node_emb.weight[777].tolist())


[-0.3172686696052551, 0.1578812301158905, -0.08087439835071564, -0.004596684593707323, -19.778369903564453, 0.026383597403764725, 0.13941125571727753, 0.12102761119604111, -12.489096641540527, 0.010141335427761078, -0.030836760997772217, 0.07849875837564468, 0.00281003350391984, -15.232320785522461, -0.15866008400917053, 0.19668158888816833, -0.19905561208724976, -0.5277314782142639, -0.08227643370628357, -0.17938721179962158, 0.015473323874175549, -0.06681554019451141, 0.11557577550411224, 0.05651519075036049, 0.2853202223777771, -0.05212356895208359, -0.1935097575187683, 0.27248385548591614, 0.0724930614233017, 0.0949799194931984, 0.08945981413125992, 0.04437510669231415, -0.1985391527414322, -18.364181518554688, -0.048563919961452484, -0.08124208450317383, -0.25457608699798584, -0.03215474635362625, -0.48660868406295776, -0.04439295083284378, 0.584082841873169, 0.403334379196167, 0.058653656393289566, 10.785226821899414, 0.1640355885028839, -0.1641729325056076, -0.07735151797533035,

In [26]:
# write embeddings to graph
for i in range(0, len(nodeId_to_id)):
    if i % 100 == 0:
        print(i)
    gds.run_cypher(
            "MATCH (n:Entity {id: $i}) SET n.emb = $EMBEDDING",
            params={
                "i": i,
                "EMBEDDING": model.node_emb.weight[i].tolist()
            },
        )

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500


In [29]:
node_projection_test = {"Entity": {"properties": ["id","emb"] }}
relationship_projection_test = {"TEST" : {"orientation": "NATURAL", "properties": "rel_id"}}
G, result = gds.graph.project(
        "fb15k-graph-test1",
        node_projection_test,
        relationship_projection_test,
    )

print(f"Graph '{G.name()}' node count: {G.node_count()}")
print(f"Graph '{G.name()}' node labels: {G.node_labels()}")
print(f"Graph '{G.name()}' relationship count: {G.relationship_count()}")

Graph 'fb15k-graph-test1' node count: 14541
Graph 'fb15k-graph-test1' node labels: ['Entity']
Graph 'fb15k-graph-test1' relationship count: 20466


In [39]:
rel_148_emb = model.node_emb.weight[148].tolist()

In [42]:
transe_model = gds.model.transe.create(
    G, "emb", {"REL": rel_148_emb}
)
result = transe_model.predict_stream(
    [id_to_nodeId[6180], id_to_nodeId[1454]], [id_to_nodeId[2861], id_to_nodeId[5951], id_to_nodeId[637]], "REL", 3, concurrency=4
)
print(result)


   sourceNodeId  targetNodeId      score
0          1521           668  66.766521
1          1521          2984  69.818382
2          1521          6187  74.640317
3          6430           668  52.112705
4          6430          6187  57.971514
5          6430          2984  61.022281
